<a href="https://colab.research.google.com/github/riyaamahajan/Bilingual-Sentiment-Analysis/blob/main/lstm_bilstm_saocmt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/codemixed-hinglish/labeled_final_test_dataset.csv
/kaggle/input/codemixed-hinglish/labeled_final_train_dataset.csv


In [ ]:
data1 = pd.read_csv('/kaggle/input/codemixed-hinglish/labeled_final_test_dataset.csv')
data2 = pd.read_csv('/kaggle/input/codemixed-hinglish/labeled_final_train_dataset.csv')

In [ ]:
df = pd.concat([data1, data2], ignore_index=True)

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

LSTM Implementation

In [ ]:
##########################LSTM IMPLEMENTATION##############################
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
#LSTM Implementation
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 14000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['Cleaned_Text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 31672 unique tokens.


In [ ]:
X = tokenizer.texts_to_sequences(df['Cleaned_Text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (16999, 250)


In [ ]:
Y = pd.get_dummies(df['Sentiment']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (16999, 3)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(13599, 250) (13599, 3)
(3400, 250) (3400, 3)


In [ ]:
# variables_for_classification=3 #change it as per your number of categories
model = Sequential()
# model.add(Dense(variables_for_classification, activation='softmax'))
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(13, activation='softmax'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_11 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_2             │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
epochs = 100
batch_size = 64
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Epoch 1/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 68s 341ms/step - accuracy: 0.3751 - loss: 1.0857 - val_accuracy: 0.5066 - val_loss: 0.9589
Epoch 2/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 65s 340ms/step - accuracy: 0.5474 - loss: 0.9118 - val_accuracy: 0.5103 - val_loss: 0.9107
Epoch 3/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 65s 339ms/step - accuracy: 0.6475 - loss: 0.8128 - val_accuracy: 0.5831 - val_loss: 0.9000
Epoch 4/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 65s 340ms/step - accuracy: 0.7948 - loss: 0.6651 - val_accuracy: 0.5779 - val_loss: 0.9428
Epoch 5/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 65s 339ms/step - accuracy: 0.8511 - loss: 0.5361 - val_accuracy: 0.5941 - val_loss: 0.9611
Epoch 6/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 66s 342ms/step - accuracy: 0.8832 - loss: 0.4497 - val_accuracy: 0.5574 - val_loss: 1.0535
Epoch 7/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 65s 339ms/step - accuracy: 0.8966 - loss: 0.4054 - val_accuracy: 0.5676 - val_loss: 1.0914
Epoch 8/100
192/192 ━━━━━━━━━━━━━━━━━━━━ 65s 340ms/step - accuracy: 0.9063 -

In [ ]:
accr = model.evaluate(X_test,Y_test)

107/107 ━━━━━━━━━━━━━━━━━━━━ 9s 86ms/step - accuracy: 0.5149 - loss: 3.2249


In [ ]:
###############################BI-DIRECTIONAL LSTM##########################################
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Embedding, Dropout, Activation, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras import initializers, regularizers, constraints, optimizers, layers


from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *
import tensorflow as tf
import os
import time
import gc
import re
import glob

In [ ]:
# Define some Global Variables
max_features = 100000 # Maximum Number of words we want to include in our dictionary
maxlen = 72 # No of words in question we want to create a sequence with
embed_size = 300# Size of word to vec embedding we are using

In [ ]:
tokenizer = Tokenizer(num_words=20000, split=' ')

tokenizer.fit_on_texts(df['Cleaned_Text'])
X = tokenizer.texts_to_sequences(df['Cleaned_Text'])
X = pad_sequences(X)
# Y = data['Sentiment']

Y = pd.get_dummies(df['Sentiment']).values
print('Shape of label tensor:', Y.shape)

# We can then create our train and test sets:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.15, random_state = 42)

# X_train, X_test, y_train, y_test = train_test_split(data['Text'], data.Sentiment, test_size=.15)

Shape of label tensor: (16999, 3)


In [ ]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(14449, 22) (14449, 3)
(2550, 22) (2550, 3)


In [ ]:
max_features = 20000
# cut texts after this number of words
# (among top max_features most common words)
maxlen = 100
batch_size = 32

In [ ]:
from keras.preprocessing import sequence
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
y_train = np.array(Y_train)
y_test = np.array(Y_test)

Pad sequences (samples x time)
x_train shape: (14449, 100)
x_test shape: (2550, 100)


In [ ]:
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(3, activation='sigmoid'))

In [ ]:
#  try using different optimizers and different optimizer configs
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=100,
#           validation_data=[x_test, y_test]
         )

Train...
Epoch 1/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.4635 - loss: 0.5969
Epoch 2/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.7387 - loss: 0.4003
Epoch 3/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.8550 - loss: 0.2600
Epoch 4/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9138 - loss: 0.1682
Epoch 5/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9426 - loss: 0.1165
Epoch 6/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9528 - loss: 0.0890
Epoch 7/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9675 - loss: 0.0641
Epoch 8/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9718 - loss: 0.0518
Epoch 9/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9768 - loss: 0.0410
Epoch 10/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9790 - loss: 0.0354
Epoch 11/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9784 - loss: 0.0326
Epoch 12/100
452/452

In [ ]:
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
accr = model.evaluate(x_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f} %'.format(accr[0],accr[1]*100))

80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5085 - loss: 3.1984
Test set
  Loss: 3.150
  Accuracy: 51.608 %
